<a href="https://colab.research.google.com/github/tak-creator/giganci/blob/main/GP_Wprowadzenie_do_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wprowadzenie do NLP w Pythonie
- Tokenizacja, Lematyzacja i stemming
- Embedding
- Podobieństwo cosinusowe
- Gra słowa: Ciepło - Zimno



## 1. Tokenizacja, lematyzacja, stemming

In [1]:
#Biblioteka NLTK
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt_tab')

text = "NLP to skrót od Natural Language Processing! Po polsku to Przetwarzanie Języka Naturalnego"
print("Zdania:", sent_tokenize(text))
print("Tokeny:", word_tokenize(text))


Zdania: ['NLP to skrót od Natural Language Processing!', 'Po polsku to Przetwarzanie Języka Naturalnego']
Tokeny: ['NLP', 'to', 'skrót', 'od', 'Natural', 'Language', 'Processing', '!', 'Po', 'polsku', 'to', 'Przetwarzanie', 'Języka', 'Naturalnego']


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
from nltk.stem import WordNetLemmatizer, SnowballStemmer
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")
nltk.download('wordnet')

words = ["Kupiłem", "Bułki", "bułeczki", "brokuły", "które","byłby","czurczakowe"]
print("{:<10} {:<10} {:<10}".format("Word", "Lemma", "Stem"))
for w in words:
    print("{:<10} {:<10} {:<10}".format(w, lemmatizer.lemmatize(w, pos='v'), stemmer.stem(w)))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Word       Lemma      Stem      
Kupiłem    Kupiłem    kupiłem   
Bułki      Bułki      bułki     
bułeczki   bułeczki   bułeczki  
brokuły    brokuły    brokułi   
które      które      które     
byłby      byłby      byłbi     
czurczakowe czurczakowe czurczakow


## 2. Embedding - słowa na wektory

In [ ]:
# Usuń wszystkie wersje scipy i numpy
!pip uninstall -y scipy numpy -q

# Zainstaluj konkretne wersje zgodne z gensim
!pip install numpy==1.26.4 scipy==1.12.0 --no-deps --force-reinstall --no-cache-dir -q

# Zainstaluj gensim w wersji 4.3.2 – stabilna i zgodna
!pip install gensim==4.3.2 --no-deps --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 260.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 226.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 277.8 MB/s eta 0:00:00


In [1]:
import os
print("Restartuję runtime")
#os.kill(os.getpid(), 9)

Restartuję runtime


In [2]:
# Tylko sprawdzenie wersji
import scipy
print("✅ SciPy version:", scipy.__version__)  # powinno dać 1.12.0

# Test ładowania modelu GloVe
import gensim
from gensim.downloader import load

print("Ładowanie modelu GloVe...")
model = load("glove-wiki-gigaword-100")



✅ SciPy version: 1.12.0
Ładowanie modelu GloVe...


In [3]:
print("dog →", model["dog"][:10])
print("dogs →", model["dogs"][:10])
print("cat →", model["cat"][:10])
print("music →", model["music"][:10])

dog → [ 0.30817  0.30938  0.52803 -0.92543 -0.73671  0.63475  0.44197  0.10262
 -0.09142 -0.56607]
dogs → [ 0.37861  0.30269  0.27838 -0.92052 -0.79141  0.75824  0.12946  0.28971
 -0.38836 -0.39586]
cat → [ 0.23088   0.28283   0.6318   -0.59411  -0.58599   0.63255   0.24402
 -0.14108   0.060815 -0.7898  ]
music → [ 0.12705   0.37253   0.23076  -0.10727   1.4273    0.7799   -0.055009
 -0.18909  -0.42828   0.28443 ]


In [9]:
slowo = input("Wpisz dowolne angielskie słowo: ").lower()

if slowo in model:
    print(f"Najbardziej podobne słowa do '{slowo}' to:")
    podobne = model.most_similar(slowo, topn=5)
    for s, podobienstwo in podobne:
        print(f"{s} (podobieństwo: {podobienstwo:.2f})")
else:
    print("To słowo nie istnieje w modelu. Spróbuj inne!")

Wpisz dowolne angielskie słowo: video
Najbardziej podobne słowa do 'video' to:
videos (podobieństwo: 0.83)
dvd (podobieństwo: 0.78)
audio (podobieństwo: 0.77)
tv (podobieństwo: 0.74)
footage (podobieństwo: 0.74)


## 3. Podobieństwo cosinusowe

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
def cos_sim(a, b):
  return cosine_similarity([a], [b])[0][0]
print("Podobieństwo 'dog' vs 'cat':", cos_sim(model['dog'], model['cat']))
print("Podobieństwo 'dog' vs 'century':", cos_sim(model['dog'], model['century']))

Podobieństwo 'dog' vs 'cat': 0.8798075
Podobieństwo 'dog' vs 'century': 0.12041345


## 4. Gra: Ciepło–Zimno

In [15]:
import random

# Zakładamy, że model i cos_sim są już załadowane:
# odel = load("glove-wiki-gigaword-50")
# def cos_sim(a, b): return cosine_similarity([a], [b])[0][0]

vocab = [w for w in model.index_to_key if w.isalpha() and len(w) > 3]
target = random.choice(vocab)

print("Zgadnij słowo (angielskie). Wpisz 'exit' by zakończyć.")

last_sim = None
wrong_guesses = 0
hint_progress = 0  # ile liter ma zostać pokazanych jako podpowiedź

while True:
    guess = input(">>> ").lower().strip()

    if guess == 'exit':
        print("Koniec gry. Słowo to:", target)
        break

    if guess == target:
        print("Brawo! Zgadłeś/-aś!")
        break

    if guess not in model:
        print("Nie znam tego słowa.")
        continue

    sim = cos_sim(model[guess], model[target])

    if last_sim is None:
        print(f"Podobieństwo: {sim:.4f}")
    else:
        print("Ciepło!" if sim > last_sim else "Zimno!", f"(sim = {sim:.4f})")

    last_sim = sim
    wrong_guesses += 1

    # Podpowiedź po 3. błędach i dalej
    if wrong_guesses >= 3:
        hint_progress = min(wrong_guesses - 2, len(target))  # nie wyjść poza długość słowa
        hint = target[:hint_progress] + "_" * (len(target) - hint_progress)
        print(f"Podpowiedź: {hint}")

Zgadnij słowo (angielskie). Wpisz 'exit' by zakończyć.
>>> hbaby
Nie znam tego słowa.
>>> baby
Podobieństwo: -0.1524
>>> babies
Ciepło! (sim = 0.0402)
>>> windows
Zimno! (sim = -0.0559)
Podpowiedź: s_______
>>> sloths
Ciepło! (sim = 0.0390)
Podpowiedź: su______
>>> sukienka
Nie znam tego słowa.
>>> exit
Koniec gry. Słowo to: sulphate


#### Wersja kodu z wbudowaną listą angielskich słówek na poziomie A2

In [14]:

# Lista słów A2
a2_words = {
    "ability", "able", "about", "above", "accident", "activity", "actor", "actress", "add", "address", "adult",
    "advice", "aeroplane", "afraid", "after", "afternoon", "again", "against", "age", "ago", "agree", "air",
    "airport", "album", "all", "all right", "almost", "along", "already", "alright", "also", "always", "amazing",
    "ambulance", "amount", "and", "angry", "animal", "another", "answer", "any", "anybody", "anyone", "anything",
    "anywhere", "apartment", "apple", "April", "area", "arm", "around", "arrive", "art", "article", "artist",
    "as", "ask", "at", "aunt", "autumn", "away", "baby", "back", "bad", "bag", "bake", "ball", "banana", "band",
    "bank", "bar", "base", "baseball", "basket", "basketball", "bath", "bathroom", "be", "bean", "bear", "beautiful",
    "because", "become", "bed", "bedroom", "beer", "before", "begin", "behind", "believe", "bell", "below", "best",
    "better", "between", "bicycle", "big", "bike", "bill", "bird", "birthday", "black", "blond", "blog", "blue",
    "board", "boat", "body", "boil", "book", "boot", "bored", "boring", "both", "bottle", "box", "boy", "boyfriend",
    "bread", "break", "breakfast", "bring", "brother", "brown", "build", "building", "bus", "business", "busy",
    "but", "butter", "buy", "by", "bye", "cafe", "cake", "call", "camera", "can", "cannot", "cap", "capital", "car",
    "card", "care", "careful", "carefully", "carry", "cat", "CD", "cent", "center", "century", "certainly", "chair",
    "change", "chart", "cheap", "check", "cheese", "chicken", "child", "chocolate", "choose", "cinema", "city",
    "class", "classmate", "classroom", "clean", "clear", "clearly", "climb", "clock", "close", "clothes", "cloud",
    "club", "coat", "coffee", "cold", "college", "color", "come", "common", "company", "compare", "complete",
    "computer", "concert", "conversation", "cook", "cooker", "cooking", "cool", "correct", "cost", "could",
    "country", "course", "cousin", "cow", "cream", "create", "cricket", "cross", "crown", "cry", "cup", "customer",
    "cut", "dad", "dance", "dancer", "dancing", "dangerous", "dark", "date", "daughter", "day", "dear", "decide",
    "deep", "delicious", "describe", "description", "design", "desk", "dictionary", "die", "different", "difficult",
    "dinner", "dirty", "discuss", "discussion", "dish", "do", "doctor", "dog", "dollar", "door", "down", "draw",
    "dream", "dress", "drink", "drive", "driver", "during", "each", "ear", "early", "east", "easy", "eat", "egg",
    "eighteen", "eighty", "either", "elephant", "eleven", "email", "end", "engine", "enjoy", "enough", "enter",
    "environment", "eraser", "euro", "even", "evening", "event", "ever", "every", "everybody", "everyone",
    "everything", "everywhere", "exam", "example", "excited", "exciting", "exercise", "expensive", "explain",
    "extra", "eye", "face", "fact", "fall", "family", "famous", "far", "farm", "farmer", "fast", "fat", "father",
    "favorite", "feel", "feeling", "festival", "few", "field", "fifteen", "fifth", "fifty", "fill", "film",
    "final", "find", "fine", "finger", "finish", "fire", "first", "fish", "five", "flat", "flight", "floor",
    "flower", "fly", "follow", "food", "foot", "football", "for", "forget", "form", "four", "fourteen", "free",
    "Friday", "friend", "friendly", "from", "front", "fruit", "full", "fun", "funny", "future", "game", "garden",
    "geography", "get", "gift", "girl", "girlfriend", "give", "glass", "glasses", "go", "good", "goodbye",
    "grandfather", "grandmother", "grape", "great", "green", "group", "grow", "guess", "guitar", "hair", "half",
    "hall", "hamburger", "hand", "happen", "happy", "hard", "hat", "hate", "have", "he", "head", "health", "healthy",
    "hear", "heart", "hello", "help", "her", "here", "hi", "high", "history", "hobby", "hold", "holiday", "home",
    "homework", "hope", "horse", "hospital", "hot", "hotel", "hour", "house", "how", "hundred", "hungry", "hurry",
    "hurt", "husband", "I", "ice", "ice cream", "idea", "if", "important", "improve", "in", "include", "information",
    "interested", "interesting", "internet", "into", "invite", "island", "it", "its", "jacket", "jam", "job", "join",
    "journey", "juice", "jump", "just", "keep", "key", "kick", "kid", "kill", "kind", "kitchen", "know", "language",
    "large", "last", "late", "later", "laugh", "lazy", "learn", "leave", "left", "leg", "lemon", "lesson", "let",
    "letter", "library", "lie", "life", "light", "like", "line", "lion", "listen", "little", "live", "long", "look",
    "lose", "lot", "love", "low", "lunch", "machine", "magazine", "main", "make", "man", "many", "map", "market",
    "married", "match", "matter", "may", "maybe", "me", "meal", "mean", "meat", "medicine", "meet", "meeting",
    "member", "message", "meter", "midnight", "milk", "million", "minute", "miss", "mistake", "model", "modern",
    "moment", "money", "monkey", "month", "moon", "more", "morning", "most", "mother", "mountain", "mouse",
    "mouth", "move", "movie", "much", "mum", "museum", "music", "must", "my", "name", "narrow", "national",
    "nature", "near", "neck", "need", "negative", "never", "new", "news", "newspaper", "next", "nice", "night",
    "no", "noise", "noisy", "north", "nose", "not", "note", "nothing", "now", "number", "nurse", "object",
    "ocean", "of", "off", "offer", "office", "often", "oh", "oil", "old", "on", "one", "only", "open", "or",
    "orange", "order", "other", "our", "out", "outside", "over", "own", "page", "paint", "pair", "paper",
    "parent", "park", "part", "partner", "party", "pass", "past", "pen", "pencil", "people", "pepper", "perfect",
    "person", "pet", "phone", "photo", "photograph", "photographer", "physics", "picture", "piece", "pig", "pilot",
    "pink", "pizza", "place", "plane", "plant", "plastic", "plate", "play", "player", "please", "plenty", "pocket",
    "police", "polite", "pool", "poor", "pop", "popular", "possible", "post", "postcard", "potato", "pound",
    "practice", "prefer", "prepare", "present", "pretty", "print", "problem", "program", "project", "pull",
    "purple", "push", "put", "quarter", "question", "quick", "quiet", "radio", "rain", "rainy", "read", "ready",
    "real", "really", "receive", "red", "remember", "repeat", "report", "rest", "restaurant", "return", "rice",
    "ride", "right", "river", "road", "room", "run", "sad", "safe", "salad", "salt", "same", "sandwich", "Saturday",
    "save", "say", "school", "science", "scientist", "score", "sea", "season", "seat", "second", "see", "sell",
    "send", "sentence", "September", "seven", "seventeen", "seventy", "several", "shake", "shall", "shampoo",
    "share", "she", "sheep", "shirt", "shoe", "shop", "shopping", "short", "should", "shoulder", "show", "sick",
    "side", "sign", "silver", "similar", "simple", "since", "sing", "singer", "sister", "sit", "six", "sixteen",
    "sixty", "size", "skate", "skateboard", "ski", "skirt", "sky", "sleep", "slow", "small", "smile", "snow",
    "so", "soap", "soccer", "social", "some", "somebody", "someone", "something", "sometimes", "son", "song",
    "soon", "sorry", "sound", "soup", "south", "space", "speak", "special", "spell", "spend", "sport", "spring",
    "stand", "star", "start", "station", "stay", "steak", "still", "stomach", "stop", "store", "story", "straight",
    "strange", "street", "strong", "student", "study", "stupid", "style", "subject", "success", "sugar", "summer",
    "sun", "Sunday", "supermarket", "sure", "surprise", "swim", "swimming", "table", "take", "talk", "tall", "tape",
    "taste", "taxi", "tea", "teach", "teacher", "team", "tell", "ten"
}

# Filtrowanie słów obecnych w modelu
vocab = [w for w in a2_words if w in model.key_to_index]
target = random.choice(vocab)

print("Zgadnij słowo. Wpisz 'exit', by zakończyć.")

# Funkcja podobieństwa
def cos_sim(a, b):
    return cosine_similarity([a], [b])[0][0]

# Zmienna pomocnicza
last_sim = None
wrong_guesses = 0
hint_progress = 0

# Główna pętla gry
while True:
    guess = input(">>> ").lower().strip()

    if guess == 'exit':
        print("Koniec gry. Szukane słowo to:", target)
        break

    if guess == target:
        print("Brawo! Zgadłeś/-aś słowo!")
        break

    if guess not in model:
        print("Nie znam tego słowa.")
        continue

    sim = cos_sim(model[guess], model[target])

    if last_sim is None:
        print(f"Podobieństwo: {sim:.4f}")
    else:
        print("Ciepło!" if sim > last_sim else "Zimno!", f"(sim = {sim:.4f})")

    last_sim = sim
    wrong_guesses += 1

    if wrong_guesses >= 3:
        hint_progress = min(wrong_guesses - 2, len(target))
        hint = target[:hint_progress] + "_" * (len(target) - hint_progress)
        print(f"Podpowiedź: {hint}")

Zgadnij słowo. Wpisz 'exit', by zakończyć.
>>> able
Podobieństwo: 0.2712
>>> wood
Ciepło! (sim = 0.3967)
>>> window
Ciepło! (sim = 0.4683)
Podpowiedź: p____
>>> 
Nie znam tego słowa.
>>> party
Zimno! (sim = 0.1572)
Podpowiedź: pl___
>>> plant
Ciepło! (sim = 0.2253)
Podpowiedź: pla__
>>> planty
Zimno! (sim = -0.1894)
Podpowiedź: plat_
>>> plants
Ciepło! (sim = 0.1913)
Podpowiedź: plate
>>> plate
Brawo! Zgadłeś/-aś słowo!


In [ ]:
# Install required libraries if not already installed
print("Installing gensim and compatible numpy/scipy versions...")
# Uninstall conflicting versions
!pip uninstall -y scipy numpy -q
# Install specific versions compatible with gensim
!pip install numpy==1.26.4 scipy==1.12.0 --no-deps --force-reinstall --no-cache-dir -q
# Install gensim
!pip install gensim==4.3.2 --no-deps --force-reinstall --no-cache-dir -q